In [1091]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [1092]:
movies = pd.read_csv('IMDB_dataset.csv', encoding='utf-16', encoding_errors='ignore', delimiter=";")
movies.head()

,IMBD title ID,Original title,Release year,Genre,Duration,Country,Content Rating,Director,Unnamed: 8,Income,Votes,Score
0,tt0111161,The Shawshank Redemption,1995-02-10,Drama,142,USA,R,Frank Darabont,NaN,$ 28815245,2.278.845,9.3
1,tt0068646,The Godfather,09 21 1972,"Crime, Drama",175,USA,R,Francis Ford Coppola,NaN,$ 246120974,1.572.674,9.2
2,tt0468569,The Dark Knight,23 -07-2008,"Action, Crime, Drama",152,US,PG-13,Christopher Nolan,NaN,$ 1005455211,2.241.615,9.
3,tt0071562,The Godfather: Part II,1975-09-25,"Crime, Drama",220,USA,R,Francis Ford Coppola,NaN,"$ 4o8,035,783",1.098.714,"9,.0"
4,tt0110912,Pulp Fiction,1994-10-28,"Crime, Drama",,USA,R,Quentin Tarantino,NaN,$ 222831817,1.780.147,"8,9f"


In [1093]:
movies[movies.duplicated()].count() #no duplicates

IMBD title ID     0
Original title    0
Release year      0
Genre             0
Duration          0
Country           0
Content Rating    0
Director          0
Unnamed: 8        0
Income            0
Votes             0
Score             0
dtype: int64

In [1094]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   IMBD title ID   100 non-null    object 
 1   Original title  100 non-null    object 
 2   Release year    100 non-null    object 
 3   Genre           100 non-null    object 
 4   Duration        99 non-null     object 
 5   Country         100 non-null    object 
 6   Content Rating  77 non-null     object 
 7   Director        100 non-null    object 
 8   Unnamed: 8      0 non-null      float64
 9   Income          100 non-null    object 
 10  Votes           100 non-null    object 
 11  Score           100 non-null    object 
dtypes: float64(1), object(11)
memory usage: 9.6+ KB


In [1095]:
movies.drop(columns=['Unnamed: 8'], inplace=True)
movies.dropna(subset=["IMBD title ID"], inplace=True)
movies["IMBD title ID"].unique()


array(['tt0111161', 'tt0068646', 'tt0468569', 'tt0071562', 'tt0110912',
       'tt0167260', 'tt0108052', 'tt0050083', 'tt1375666', 'tt0137523',
       'tt0109830', 'tt0120737', 'tt0060196', 'tt0133093', 'tt0167261',
       'tt0080684', 'tt0099685', 'tt0073486', 'tt0816692', 'tt0114369',
       'tt0102926', 'tt0076759', 'tt0120815', 'tt0120689', 'tt0317248',
       'tt0245429', 'tt0118799', 'tt6751668', 'tt0038650', 'tt0047478',
       'tt0172495', 'tt0407887', 'tt0482571', 'tt0088763', 'tt0120586',
       'tt0110413', 'tt0103064', 'tt0114814', 'tt0110357', 'tt7286456',
       'tt1675434', 'tt0253474', 'tt2582802', 'tt0054215', 'tt0034583',
       'tt0064116', 'tt0095327', 'tt0095765', 'tt0027977', 'tt1345836',
       'tt1853728', 'tt0209144', 'tt0910970', 'tt0081505', 'tt0082971',
       'tt4154756', 'tt0078748', 'tt4154796', 'tt0078788', 'tt0364569',
       'tt0057012', 'tt0047396', 'tt2380307', 'tt0405094', 'tt4633694',
       'tt1187043', 'tt0119698', 'tt0087843', 'tt0032553', 'tt00

In [1097]:
movies["Release year"].unique()

array(['1995-02-10', '09 21 1972', ' 23 -07-2008', '1975-09-25',
       '1994-10-28', '22 Feb 04', '1994-03-11', '1957-09-04',
       '2010-09-24', '10-29-99', '1994-10-06', '2002-01-18',
       '23rd December of 1966 ', '1999-05-07', '01/16-03', '1980-09-19',
       '1990-09-20', '18/11/1976', '2014-11-06', '1995-12-15',
       '1991-03-05', '1977-10-20', '1998-10-30', '2000-10-03',
       '2003-05-09', '2003-04-18', '1997-12-20', '2019-11-07',
       '1948-03-11', '1955-08-19', '2000-05-19', '2006-10-27',
       '2006-12-22', '1985-10-18', '1999-08-27', '1995-04-07',
       '1991-12-19', '1995-11-30', '1994-11-25', '2019-10-03',
       '2012-02-24', '2002-10-25', '2015-02-12', '1960-10-28', '21-11-46',
       '1968-12-21', '2015-10-11', '1988-11-17', '1937-03-12',
       '2012-08-29', '2013-01-17', '2001-01-19', '2008-10-17',
       '1980-12-22', '1981-06-12', '2018-04-25', '1979-10-25',
       '2019-04-24', '1979-12-18', '2005-05-06', '1964-04-03',
       '1955-04-14', '2017-12-28',

In [1098]:
movies["Release year"] = movies["Release year"].str.strip().str.replace("/","-").str.replace(" ", "-").str.replace("--", "-").str.replace("23rd-December-of-1966", "1966-12-23").str.replace("The-6th-of-marzo,-year-1951", "1951-03-06").str.replace("Feb", "02")
print(movies["Release year"].unique())
def parse_mixed_dates(date_str):
    formats = ["%Y-%m-%d", "%d-%m-%Y", "%m-%d-%Y"]
    
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            continue  # If parsing fails, try the next format
    
    return pd.NaT
movies["Release year"] = movies["Release year"].apply(parse_mixed_dates)
mode = movies["Release year"].mode()[0]
movies["Release year"]=movies["Release year"].fillna(mode)
movies["Release year"].value_counts()
movies["Release year"] = pd.to_datetime(movies["Release year"]).dt.year
movies.info()

['1995-02-10' '09-21-1972' '23-07-2008' '1975-09-25' '1994-10-28'
 '22-02-04' '1994-03-11' '1957-09-04' '2010-09-24' '10-29-99' '1994-10-06'
 '2002-01-18' '1966-12-23' '1999-05-07' '01-16-03' '1980-09-19'
 '1990-09-20' '18-11-1976' '2014-11-06' '1995-12-15' '1991-03-05'
 '1977-10-20' '1998-10-30' '2000-10-03' '2003-05-09' '2003-04-18'
 '1997-12-20' '2019-11-07' '1948-03-11' '1955-08-19' '2000-05-19'
 '2006-10-27' '2006-12-22' '1985-10-18' '1999-08-27' '1995-04-07'
 '1991-12-19' '1995-11-30' '1994-11-25' '2019-10-03' '2012-02-24'
 '2002-10-25' '2015-02-12' '1960-10-28' '21-11-46' '1968-12-21'
 '2015-10-11' '1988-11-17' '1937-03-12' '2012-08-29' '2013-01-17'
 '2001-01-19' '2008-10-17' '1980-12-22' '1981-06-12' '2018-04-25'
 '1979-10-25' '2019-04-24' '1979-12-18' '2005-05-06' '1964-04-03'
 '1955-04-14' '2017-12-28' '2007-04-06' '2018-12-25' '2009-12-25'
 '1984-09-28' '1945-06-25' '1951-03-06' '2009-10-02' '2000-01-21'
 '1995-12-01' '1983-10-21' '1992-10-09' '2004-10-22' '1996-03-22'
 '199

In [1099]:
movies["Genre"].unique()
# Split the 'Genre' column by comma
split_genre = movies['Genre'].str.split(',', expand=True)

unique_values = pd.unique(split_genre.values.ravel())
unique_values = [val.strip() for val in unique_values if pd.notna(val)]
print(unique_values)

# Step 3: One-Hot Encoding (Create new columns for each unique value)
for val in unique_values:
    movies[val] = movies['Genre'].apply(lambda x: 1 if val in x else 0)

# Display the updated DataFrame
movies.drop(columns=['Genre'], inplace=True)
movies.head()

['Drama', 'Crime', 'Drama', 'Action', 'Crime', 'Adventure', 'Biography', 'History', 'Sci-Fi', 'Romance', 'Western', 'Fantasy', 'Adventure', 'Mystery', 'Thriller', 'War', 'Animation', 'Family', 'Comedy', 'Comedy', 'Music', 'Horror', 'Western', 'Mystery', 'Horror', 'Action', 'Film-Noir', 'Biography', 'Musical']


,IMBD title ID,Original title,Release year,Duration,Country,Content Rating,Director,Income,Votes,Score,...,Mystery,Thriller,War,Animation,Family,Comedy,Music,Horror,Film-Noir,Musical
0,tt0111161,The Shawshank Redemption,1995,142,USA,R,Frank Darabont,$ 28815245,2.278.845,9.3,...,0,0,0,0,0,0,0,0,0,0
1,tt0068646,The Godfather,1972,175,USA,R,Francis Ford Coppola,$ 246120974,1.572.674,9.2,...,0,0,0,0,0,0,0,0,0,0
2,tt0468569,The Dark Knight,2008,152,US,PG-13,Christopher Nolan,$ 1005455211,2.241.615,9.,...,0,0,0,0,0,0,0,0,0,0
3,tt0071562,The Godfather: Part II,1975,220,USA,R,Francis Ford Coppola,"$ 4o8,035,783",1.098.714,"9,.0",...,0,0,0,0,0,0,0,0,0,0
4,tt0110912,Pulp Fiction,1994,135,USA,R,Quentin Tarantino,$ 222831817,1.780.147,"8,9f",...,0,0,0,0,0,0,0,0,0,0


In [1096]:
movies["Duration"].unique()
movies["Duration"] = movies["Duration"].str.strip(" c-InfNot Applicable")
movies["Duration"].unique()
movies.loc[movies["Duration"].isin([""]), "Duration"] = np.nan
movies["Duration"].unique()
movies["Duration"] = movies["Duration"].astype(float)
movies["Duration"]=movies["Duration"].fillna(movies["Duration"].mean().round(0))
movies["Duration"] = movies["Duration"].astype(int)
movies["Duration"].unique()

array([142, 175, 152, 220, 135, 201,  96, 148, 178, 161, 179, 146, 169,
       127, 118, 121, 189, 130, 125, 116, 132, 207, 155, 151, 119, 110,
       137, 106,  88, 122, 112, 150, 109, 102, 165,  89,  87, 164, 113,
        98, 115, 149, 117, 181, 147, 120,  95, 105, 170, 134, 229, 153,
       131,  99, 108,  81, 126, 104, 136, 103, 114, 160, 128, 228, 129,
       123])

In [1100]:
movies.loc[movies["Country"].str.contains("US"), "Country"] = "USA"
movies.loc[movies["Country"].str.contains("Germany"), "Country"] = "Germany"
movies.loc[movies["Country"].str.contains("New Z"), "Country"] = "New Zealand"
movies.loc[movies["Country"].str.contains("Italy"), "Country"] = "Italy"
movies["Country"].unique()

array(['USA', 'New Zealand', 'UK', 'Italy', 'Brazil', 'Japan',
       'South Korea', 'France', 'Germany', 'India', 'Denmark', 'Iran'],
      dtype=object)

In [1101]:
movies["Content Rating"].unique()
movies["Content Rating"] = movies["Content Rating"].fillna("Not Rated")
movies.loc[movies["Content Rating"].str.contains("Not Rated|Approved|Unrated"), "Content Rating"] = "Not Rated"
movies["Content Rating"] = movies["Content Rating"].astype("category")
movies["Content Rating"] = movies["Content Rating"].cat.set_categories(new_categories=["Not Rated", "G", "PG", "PG-13", "R"], ordered=True)
movies["Content Rating"].value_counts()

Content Rating
R            45
Not Rated    26
PG-13        12
PG           11
G             6
Name: count, dtype: int64

In [1102]:
movies["Income"].unique()
movies["Income"]=movies["Income"].str.strip("$ ").str.replace(",","").str.replace("o","0").astype(int)
movies["Income"].unique()

array([  28815245,  246120974, 1005455211,  408035783,  222831817,
       1142271098,  322287794,        576,  869784991,  101218804,
        678229452,  887934303,   25252481,  465718588,  951227416,
        549265501,   46879633,  108997629,  696742056,  327333559,
        272753884,  775768912,  482349603,  286801374,   30680793,
        355467056,  230098753,  257604912,    6130720,     322773,
        465361176,  291465034,  109676311,  388774684,   23875127,
         19552639,  520884847,   23341568,  968511805, 1074251311,
        426588510,  120072577,   48983260,   32008644,    4374761,
           112911,     516962,   13826605,     457688, 1081133191,
        425368238,   39970386,  521311860,   46520613,  390133212,
       2048359754,  108110316, 2797800564,   91968688,   15002116,
          9443876,   37032034,  807083670,   77356942,  375540831,
         60262836,  169785629,    5472914,     969879,     299645,
        321455689,  356296601,  213216216,  475347111,    2889

In [1103]:
movies["Votes"].unique()
movies["Votes"]=movies["Votes"].str.replace(".","").astype(int)
movies["Votes"].unique()

array([2278845, 1572674, 2241615, 1098714, 1780147, 1604280, 1183248,
        668473, 2002816, 1807440, 1755490, 1619920,  672499, 1632315,
       1449778, 1132073,  991505,  891071, 1449256, 1402015, 1234134,
       1204107, 1203825, 1112336,  685856,  626693,  605648,  470931,
        388310,  307958, 1308193, 1159703, 1155723, 1027330, 1014218,
       1007598,  974970,  968947,  917248,  855097,  736691,  707942,
        690732,  586765,  509953,  295220,  225438,  223050,  211250,
       1480582, 1317856, 1098879,  974734,  869480,  865510,  796486,
        768874,  754786,  591251,  501082,  441115,  432390,  352455,
        349642,  335892,  332217,  331045,  302317,  197381,  195789,
       1229958, 1049009,  941683,  928036,  896551,  889875,  864461,
        837379,  766589,  748291,  740301,  739717,  721343,  703264,
        690480,  658175,  639500,  587866,  389322,  362897,  361028,
        352786,  291628,  269616,  261504,  236285,  226427,  226039,
        214165,  213

In [1104]:
movies["Score"].unique()
movies["Score"]=movies["Score"].str.strip("0.e-0++f").str.replace(",",".").str.replace(":",".")
movies.loc[movies["Score"].str.contains("9."), "Score"] = "9.0"
movies.loc[movies["Score"].str.contains("8..8"), "Score"] = "8.8"
movies["Score"]=movies["Score"].astype(float)
movies["Score"].unique()

array([9. , 8.9, 8.8, 8.7, 8.6, 8.5, 8.4, 8.3, 8.2, 8.1, 8. , 7.9, 7.8,
       7.7, 7.6, 7.5, 7.4])

In [1105]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 0 to 100
Data columns (total 30 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   IMBD title ID   100 non-null    object  
 1   Original title  100 non-null    object  
 2   Release year    100 non-null    int32   
 3   Duration        100 non-null    int64   
 4   Country         100 non-null    object  
 5   Content Rating  100 non-null    category
 6   Director        100 non-null    object  
 7   Income          100 non-null    int64   
 8   Votes           100 non-null    int64   
 9   Score           100 non-null    float64 
 10  Drama           100 non-null    int64   
 11  Crime           100 non-null    int64   
 12  Action          100 non-null    int64   
 13  Adventure       100 non-null    int64   
 14  Biography       100 non-null    int64   
 15  History         100 non-null    int64   
 16  Sci-Fi          100 non-null    int64   
 17  Romance         100 n

In [1106]:
movies.head()

,IMBD title ID,Original title,Release year,Duration,Country,Content Rating,Director,Income,Votes,Score,...,Mystery,Thriller,War,Animation,Family,Comedy,Music,Horror,Film-Noir,Musical
0,tt0111161,The Shawshank Redemption,1995,142,USA,R,Frank Darabont,28815245,2278845,9.0,...,0,0,0,0,0,0,0,0,0,0
1,tt0068646,The Godfather,1972,175,USA,R,Francis Ford Coppola,246120974,1572674,9.0,...,0,0,0,0,0,0,0,0,0,0
2,tt0468569,The Dark Knight,2008,152,USA,PG-13,Christopher Nolan,1005455211,2241615,9.0,...,0,0,0,0,0,0,0,0,0,0
3,tt0071562,The Godfather: Part II,1975,220,USA,R,Francis Ford Coppola,408035783,1098714,9.0,...,0,0,0,0,0,0,0,0,0,0
4,tt0110912,Pulp Fiction,1994,135,USA,R,Quentin Tarantino,222831817,1780147,8.9,...,0,0,0,0,0,0,0,0,0,0
